#**Text Classification with the IMDb-Reviews Dataset from Keras**
@author: [vatsalya-gupta](https://github.com/vatsalya-gupta)

In [0]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

data = keras.datasets.imdb

We will do our analysis with the 85000 most frequent unique words.

In [2]:
(train_data, train_labels), (test_data, test_labels) = data.load_data(num_words = 85000)
print(train_data[0])

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 22665, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 21631, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 19193, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 10311, 8, 4, 107, 117, 5952, 15, 256, 4, 31050, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 12118, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]


Our training and testing data is in the form of an array of reviews, where each review is a list of integers and each integer represents a unique word. So we need to make it human readable. For this, we will be adding the following tags to the data, map the values to their respective keys and implement a function which converts the integers to the respective words.

In [0]:
word_index = data.get_word_index()
word_index = {k: (v + 3) for k, v in word_index.items()}
word_index["<PAD>"] = 0
word_index["<START>"] = 1
word_index["<UNK>"] = 2    # unknown
word_index["<UNUSED>"] = 3

reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

In [0]:
def decode_review(text):
	return " ".join([reverse_word_index.get(i, "?") for i in text])

In [5]:
print(decode_review(train_data[0]))

<START> this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert redford's is an amazing actor and now the same being director norman's father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for retail and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also congratulations to the two little boy's that played the part's of norman and paul they were just brilliant children are often left out of the praising list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and sho

In [6]:
print(len(train_data[0]), len(test_data[0]))

218 68


In the following block of code, we will be finding the length of the longest review in our dataset.

In [7]:
longest_train = max(len(l) for l in train_data)
longest_test = max(len(l) for l in test_data)

max_words = max(longest_train, longest_test)
print(max_words)

2494


Even though the longest review is 2494 words long, we can safely limit the length of our reviews to 250 words as most of them are well below that. For the ones with length less than 250 words, we need to add padding to their end, as our model requires the review length to be 250 words.

In [8]:
train_data = keras.preprocessing.sequence.pad_sequences(train_data, value = word_index["<PAD>"], padding = "post", maxlen = 250)
test_data = keras.preprocessing.sequence.pad_sequences(test_data, value = word_index["<PAD>"], padding = "post", maxlen = 250)

print(len(train_data[0]), len(test_data[0]))

250 250


We are using a Sequential model. An Embedding layer attempts to determine the meaning of each word in the sentence by mapping each word to a position in vector space (helps in grouping words like "fantastic" and "awesome"). The GlobalAveragePooling1D layer scales down our data's dimensions to make it easier computationally. The last two layers in our network are dense fully connected layers. The output layer is one neuron that uses the sigmoid function to get a value between 0 and 1 which will represent the likelihood of the review being negative or positive respectively.

In [9]:
model = keras.Sequential()
model.add(keras.layers.Embedding(85000, 16))
model.add(keras.layers.GlobalAveragePooling1D())
model.add(keras.layers.Dense(16, activation = "relu"))
model.add(keras.layers.Dense(1, activation = "sigmoid"))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 16)          1360000   
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 16)                272       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 1,360,289
Trainable params: 1,360,289
Non-trainable params: 0
_________________________________________________________________


Compiling the data using the following parameters. We are using loss as "binary_crossentropy", as the expected output of our model is either 0 or 1, that is negative or positive.

In [0]:
model.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = ["accuracy"])

Here we split the training data into training and validation sets.

In [0]:
X_val = train_data[:10000]
X_train = train_data[10000:]

y_val = train_labels[:10000]
y_train = train_labels[10000:]

The training data is fit onto the model and the results are evaluated.

In [12]:
fitModel = model.fit(X_train, y_train, epochs = 40, batch_size = 512, validation_data = (X_val, y_val), verbose = 1)
results = model.evaluate(test_data, test_labels)

print(results)

Epoch 1/40
30/30 [==============================] - 1s 37ms/step - loss: 0.6917 - accuracy: 0.5568 - val_loss: 0.6892 - val_accuracy: 0.7054
Epoch 2/40
30/30 [==============================] - 1s 29ms/step - loss: 0.6845 - accuracy: 0.7393 - val_loss: 0.6795 - val_accuracy: 0.7442
Epoch 3/40
30/30 [==============================] - 1s 28ms/step - loss: 0.6695 - accuracy: 0.7751 - val_loss: 0.6612 - val_accuracy: 0.7383
Epoch 4/40
30/30 [==============================] - 1s 28ms/step - loss: 0.6430 - accuracy: 0.7712 - val_loss: 0.6324 - val_accuracy: 0.7866
Epoch 5/40
30/30 [==============================] - 1s 28ms/step - loss: 0.6046 - accuracy: 0.8037 - val_loss: 0.5947 - val_accuracy: 0.8019
Epoch 6/40
30/30 [==============================] - 1s 28ms/step - loss: 0.5571 - accuracy: 0.8281 - val_loss: 0.5507 - val_accuracy: 0.8131
Epoch 7/40
30/30 [==============================] - 1s 28ms/step - loss: 0.5043 - accuracy: 0.8521 - val_loss: 0.5052 - val_accuracy: 0.8231
Epoch 8/40
30

Sample prediction from testing data.

In [13]:
test_review = test_data[0]
predict = model.predict([test_review])
print("Review:\n", decode_review(test_review))
print("Prediction:", predict[0])
print("Actual:", test_labels[0])

Review:
 <START> please give this one a miss br br kristy swanson and the rest of the cast rendered terrible performances the show is flat flat flat br br i don't know how michael madison could have allowed this one on his plate he almost seemed to know this wasn't going to work out and his performance was quite lacklustre so all you madison fans give this a miss <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD

Saving the model so that we don't have to train it again.

In [0]:
model.save("imdb_model.h5")    # any name ending with .h5
# model = keras.models.load_model("imdb_model.h5")    # loading the model, use this in any other project for testing

Function to encode a text based review into a list of integers.

In [0]:
def review_encode(s):
	encoded = [1]

	for word in s:
		if word.lower() in word_index:
			encoded.append(word_index[word.lower()])
		else:
			encoded.append(2)

	return encoded

Evaluating our model on an [external review](https://www.imdb.com/review/rw2284594).

In [16]:
with open("sample_data/test.txt", encoding = "utf-8") as f:
	for line in f.readlines():
		nline = line.replace(",", "").replace(".", "").replace("(", "").replace(")", "").replace(":", "").replace("\"","").strip().split(" ")
		encode = review_encode(nline)
		encode = keras.preprocessing.sequence.pad_sequences([encode], value = word_index["<PAD>"], padding = "post", maxlen = 250)    # make the data 250 words long
		predict = model.predict(encode)
		print(line, "\n", encode, "\n", predict[0])

The Shawshank Redemption is written and directed by Frank Darabont. It is an adaptation of the Stephen King novella Rita Hayworth and Shawshank Redemption. Starring Tim Robbins and Morgan Freeman, the film portrays the story of Andy Dufresne (Robbins), a banker who is sentenced to two life sentences at Shawshank State Prison for apparently murdering his wife and her lover. Andy finds it tough going but finds solace in the friendship he forms with fellow inmate Ellis "Red" Redding (Freeman). While things start to pick up when the warden finds Andy a prison job more befitting his talents as a banker. However, the arrival of another inmate is going to vastly change things for all of them. There was no fanfare or bunting put out for the release of the film back in 94, with a title that didn't give much inkling to anyone about what it was about, and with Columbia Pictures unsure how to market it, Shawshank Redemption barely registered at the box office. However, come Academy Award time the 

We are able to achieve a score of "highly positive" on the review rated 10/10 on IMDb. Hence, our model is fairly accurate.